___
# Ciência dos Dados - PROJETO 2

___
## Beni Stern

## Leonardo Costa

## Gabriel Duarte

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [12]:
import pandas as pd
import os
import numpy as np

In [13]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Beni Stern\Desktop\CDD\Projeto-2-Filtro-Spam


In [22]:
dados = pd.read_excel("spamham2019.xlsx")
dados

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
5,FreeMsg Hey there darling it's been 3 week's n...,spam
6,Even my brother is not like to speak with me. ...,ham
7,As per your request 'Melle Melle (Oru Minnamin...,ham
8,WINNER!! As a valued network customer you have...,spam
9,Had your mobile 11 months or more? U R entitle...,spam


In [25]:
dados_aleatorio = dados.sample(frac=1)
dados_aleatorio = dados.sample(frac=1).reset_index(drop=True)
dados_75 = dados_aleatorio.loc[(dados_aleatorio.index<=4180),:]
dados_25 = dados_aleatorio.loc[(dados_aleatorio.index>=4180),:]
dados_aleatorio.head()

,Email,Class
0,FREE for 1st week! No1 Nokia tone 4 ur mobile ...,spam
1,R we going with the &lt;#&gt; bus?,ham
2,Well done and ! luv ya all,ham
3,He fucking chickened out. He messaged me he wo...,ham
4,Im just wondering what your doing right now?,ham


In [6]:
dados_75.head()

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [26]:
dados_25.head()

,Email,Class
4180,"Sorry, I'll call later",ham
4181,R u over scratching it?,ham
4182,Ok i am on the way to home hi hi,ham
4183,U have a secret admirer. REVEAL who thinks U R...,spam
4184,We still on for tonight?,ham


___
## 3. Classificador Naive-Bayes

___
## 4. Qualidade do Classificador alterando a base de treinamento